# Importando bibliotecas

In [ ]:
import pandas as pd
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# Lista dos criadores
 Utilizando a biblioteca Selenium para acessar a lista dos 50 criadores com maior volume de vendas no período de 7 dias

In [ ]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.implicitly_wait(1.5)
driver.get('https://www.finder.com.au/cryptocurrency/nft-collections')

In [ ]:
link_html = driver.find_element(By.XPATH,'''//*[@id="main"]/div[4]/div[2]/div[2]/div/div[2]/ul/li[3]/a''')

In [ ]:
link_html.click()

In [ ]:
lista_name = []
lista_sales =[]
lista_volume =[]
for i in range(1,26):
    lista_name.append(driver.find_element(By.XPATH, f'''//*[@id="pb-tabname2"]/div/table/tbody/tr[{i}]/td[2]''').text)
    lista_sales.append(driver.find_element(By.XPATH, f'''//*[@id="pb-tabname2"]/div/table/tbody/tr[{i}]/td[3]''').text)
    lista_volume.append(re.sub(' [A-Z]+','',driver.find_element(By.XPATH, f'''//*[@id="pb-tabname2"]/div/table/tbody/tr[{i}]/td[4]''').text))

Criando o DataFrame com as informações dos criadores com o volume de negociações em quantidade de vendas e em Ethereum

In [ ]:
df1 = pd.DataFrame({'creator':lista_name , 'sales':lista_sales, 'volume [ETH]':lista_volume} )

In [ ]:
df1

# DataFrame vendas discretizadas


Criando listas para armazenar os dados

In [ ]:
lista_address_aux1 = []
lista_creators1 = []
lista_tokenid1 = []
lista_price1 = []
lista_block1 = []
lista_block_minter1 = []
lista_contract1 = []

Looping utilizando a lista obtida anteriormente, com auxilio de Webscrapping e API para extrair os dados em formato de lista para posteriormente criar o DataFrame

In [ ]:
for i in lista_name:
    driver = webdriver.Chrome(service=s)
    driver.implicitly_wait(5)
    driver.get('https://opensea.io/')
    driver.implicitly_wait(10)
    buscador = driver.find_element(By.XPATH, '''//*[@id="__next"]/div/div[1]/nav/div[2]/div/div/div/input''')
    buscador.send_keys(i)
    time.sleep(5)
    primeiro_item = driver.find_element(By.XPATH,f'''//*[contains(text(),'{i}')]''')
    primeiro_item.click()
    time.sleep(5)
    primeiro_nft = driver.find_element(By.XPATH,'''//*[@id="main"]/div/div/div[3]/div/div/div/div[3]/div[3]/div[2]/div/div/div[1]/div/article/a/div[2]''')
    primeiro_nft.click()
    address = driver.current_url
    contract = re.findall('[0-9]\w+',address)
    driver.get('https://opensea.io/')
    
    endpoint = f'https://deep-index.moralis.io/api/v2/nft/{contract[0]}/trades'
    query = {'chain':'eth','marketplace':'opensea','from_date':'01/04/2022','to_date':'02/04/2022'}
    header = {'accept' : 'application/json', 'X-API-Key' : 'zI2h59sDw8RlpIkIierrdsqUJVD8GvU3bu6UcxwvI4NQLBoj1qazAqSf0ugb6iZa'}
    resposta = requests.request('GET', endpoint, headers=header, params=query)
    resultados = resposta.json()
    time.sleep(10)
    for items in resultados['result']:
        lista_tokenid1.append(items['token_ids'][0])
        lista_price1.append(items['price'])
        lista_block1.append(items['block_number'])
        lista_address_aux1.append(contract[0])
        lista_creators1.append(i)
        endpoint2 = f"https://deep-index.moralis.io/api/v2/nft/{contract[0]}/{items['token_ids'][0]}"
        query2 = {'chain':'eth', 'format':'decimal'}
        header2 = {'accept' : 'application/json', 'X-API-Key' : 'zI2h59sDw8RlpIkIierrdsqUJVD8GvU3bu6UcxwvI4NQLBoj1qazAqSf0ugb6iZa'}
        response = requests.request('GET', endpoint2, headers=header2, params=query2)
        results = response.json()
        try:
            lista_block_minter1.append(results['block_number_minted'])
        except KeyError:
            lista_block_minter1.isna()
        try:
            lista_contract1.append(results['contract_type'])
        except KeyError:
            lista_contract1.isna()
            

Criação do Dataframe 

In [ ]:
df2 = pd.DataFrame({'Criador':lista_creators1, 'Contract Address':lista_address_aux1, 'Token Id': lista_tokenid1,'Preço' :lista_price1,'Block Chain': lista_block1})
df2

Exportando como .csv para criação do dashboard via Tableau

In [ ]:
df2.to_csv('C:\Users\guede\projetos\projeto_extracao_dados_nft/df_nft.csv')